$$\Huge{\textbf{CLOUDS Assignment 2}}$$

This assignment is a series of labs using the Azure cloud. In the last assignment, you covered
material from lectures 1 and 2. In this assignment, you will carry out a series of tasks that cover
lectures 3 and 4.

**NOTE: All the code you write in this assignment should be stored in a git repository. I
suggest you one repo for all your assignments with separate folders per assignment. You
can use gitlab.eurecom.fr. You will need to submit your gitlab link.**

# 1. Create a microservice that does numerical integration

1. We saw how to do numerical integration in the class. You first task is to create a program that can
do numerical integration with the function abs(sin(x)). Given an interval lower and upper, your code
should break up the interval into $N$ subintervals, compute the area of the rectangle at each
subinterval and add them all up. For example, if you give as input 0 and 3.14159 (which is
approximately $\pi$), you should get $1.99…$ which is close to $2$ ($\int \sin 𝑥 = − \cos 𝑥$, ignoring aspects of
continuity, which when evaluated in the range gives you $- \cos \pi + \cos 0 = 2$). Your program should
loop and repeatedly compute numerical integral for $N = 10, 100, 100, 1000, 10k, 100k, 1M$. You will
end up getting 7 values, one for each value of N. You will see that as N increases, result
converges to 2.This will also make the integral computation time consuming which will be useful for
load testing later.

2. Now take the python program from above and convert it into a microservice. Read section 19.1 if
you want to know what a microservice is if you have not read it already. You can use any
framework you want (python (flask), javascript (nodejs), C# (.net)). I recommend python & flask.
The microservice, when run, will take the lower and upper as arguments. These could be URL
route or parameters (http://localhost:5000/numericalintegralservice/0/3.14159). What you are
doing is instead of invoking the program via python interpreter, you are invoking it via a HTTP
call.

3. Test your microservice locally on your computer. If you cant use a local computer, you can get a
VM from Azure and do your development there. Use curl or a browser to verify that it works.
Next, load test your microservice with Locust (https://locust.io/). The goal is to create many
clients that all invoke the service over HTTP simulatenously. Test with 1 client. If it works, test
with multiple clients. Here’s a reference guide to run locust without a UI1
(https://docs.locust.io/en/stable/running-without-web-ui.html#running-without-web-ui). You can
use a simple locustfile.py like the following:
```python
import time
from locust import HttpUser, task, between

class QuickstartUser(HttpUser):
    @task
    def hello_world(self):
        self.client.get("/integral/0/3.14159")
```

4. **For your deliverable, run Locust for 3 minutes and save its output. You will have stable
performance. You will use the output to plot a graph later.**

# 2. Improving availability with scalesets

1. Read AzureMol book chapter 5 to know about Networking (vnics mainly and ssh-agent). Read
chapter 6 to know about ARM templates, chapte 7 about availability, and chapter 8 about load
balancing. You don’t have to do the exercises if you don’t want to. Now read chapter 9 with
particular focus on VM scaling. You will see how all the above come together.
2. Create a VM scaleset with 2 VMs. Manually configure both VMs so that you the microservice you
built in part 1 is deployed on both VMs. Note that the VMs will be behind a load balancer this
time. You will be tunneling (all of this is explained in the book). Here’s a reference for a quick and
dirty way to configure your VMs to deploy a flask app with NGINX
(https://krishansubudhi.github.io/webapp/2018/12/01/flaskwebapp.html)
3. Use your development VM or your computer to now perform a load testing on the scale set you
just deployed. While the load tester is running, check Azure Insights or VM directly to see which
VM is serving the load. Note the latencies reported by your load tester also.
4. While the load tester is running, manually shutdown the VM that is servicing the load. Notice
what happens with the load tester – does throughput drop? Does it go back up again? Notice
how load balancer redirects load to the other VM.
5. **NOTE: You will need to run Locust for 3 minutes and save its output. In the middle of this
run, you will turn off the VM. You will use the output to plot a graph later.**
6. Delete the scaleset and free up all resources